In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
df=pd.read_csv(r"B:\project\newdataset\chunk_8.csv")
df = df.drop(columns=['Unnamed: 0'])
df

,text,generated
0,"Dear Principal,\n\nHello! I hope you are havin...",0.0
1,"Dear Florida senator,\n\nI strongly agree of k...",0.0
2,"Dear Senator,\n\nPresidential elections are cl...",0.0
3,"Dear Mr. Senator, Personal I believe we should...",0.0
4,"According to the article, ""Driverless Cars Are...",0.0
...,...,...
17983,There are many advantages of limiting car usag...,0.0
17984,"Dear TEACHER_NAME,\n\nI think that it is a gre...",0.0
17985,It is good to ask for advice from multiple peo...,0.0
17986,School is something that almost everyone goes ...,0.0


In [3]:
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
essays=df['text'].tolist()
tokens=tokenizer(essays,return_tensors='pt',padding=True,truncation=True)

In [4]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
input_ids_size = tokens['input_ids'].size()
attention_mask_size = tokens['attention_mask'].size()
print("Input IDs size:", input_ids_size)
print("Attention Mask size:", attention_mask_size)
# Assuming 'tokens' is the result of tokenizer
unique_tokens = set(token.item() for seq in tokens['input_ids'] for token in seq)
vocabulary_size = len(unique_tokens)

print("Vocabulary Size:", vocabulary_size)
# Assuming 'input_ids' is the first element in xtrain
max_sequence_length_x = max(len(seq) for seq in xtrain)

# Print or use the max_sequence_length_x in your pad_sequences and Embedding layer
print("Max Sequence Length:", max_sequence_length_x)


Input IDs size: torch.Size([17988, 1984])
Attention Mask size: torch.Size([17988, 1984])
Vocabulary Size: 32798
Max Sequence Length: 1984


In [5]:
model=Sequential()
input_length=1984
xtrain_np=xtrain.numpy()
model.add(Embedding(input_dim=52000,output_dim=100,input_length=input_length))
model.add(LSTM(64))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xtrain_np,ytrain,epochs=5,batch_size=32)



Epoch 1/5


450/450 [==============================] - 196s 432ms/step - loss: 0.0116 - accuracy: 1.0000
Epoch 2/5
450/450 [==============================] - 203s 451ms/step - loss: 4.0705e-05 - accuracy: 1.0000
Epoch 3/5
450/450 [==============================] - 202s 448ms/step - loss: 2.0825e-05 - accuracy: 1.0000
Epoch 4/5
450/450 [==============================] - 189s 420ms/step - loss: 1.2665e-05 - accuracy: 1.0000
Epoch 5/5
450/450 [==============================] - 192s 426ms/step - loss: 8.1903e-06 - accuracy: 1.0000


In [6]:
xtest_np=xtest.numpy()
model.evaluate(xtest_np,ytest)

113/113 [==============================] - 16s 136ms/step - loss: 6.4831e-06 - accuracy: 1.0000


[6.48305149297812e-06, 1.0]

In [7]:
model.save("newmodel8.keras")